In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
import time
import pickle

In [2]:
range_radius = np.arange(5,34,2)
conc_interval = 0.01
range_sum1 = np.linspace(0,1,int(1/conc_interval +1))
range_sum2 = np.linspace(0,1,int(1/conc_interval +1))

In [3]:
#df = pd.DataFrame(columns=['yb_1','er_1', 'yb_2','er_2', 'radius'])
def generate_grid(range_radius,range_sum1, range_sum2):
    pool = []
    for r in range_radius:
        print(f'radius = {r}')
        for i in range_sum1:
            #print(f'sum1 = {i}')
            for j in range_sum2:
                sum1 = i
                yb_1_num = int(sum1/conc_interval + 1)
                sum2 = j
                yb_2_num = int(sum2/conc_interval + 1)
                range_yb1 = np.linspace(0,sum1, yb_1_num)
                range_yb2 = np.linspace(0,sum2, yb_2_num)
                for yb1 in range_yb1:
                    for yb2 in range_yb2:
                        #df.loc[len(df.index)] = [yb1, sum1-yb1, yb2, sum2-yb2, r]
                        pool.append([yb1, sum1-yb1, yb2, sum2-yb2, r])
    return pool
#df = pd.DataFrame(columns=['yb_1','er_1', 'yb_2','er_2', 'radius'])
def generate_grid_encoded(range_radius,range_sum1, range_sum2):
    pool = []
    for r in range_radius:
        print(f'radius = {r}')
        for i in range_sum1:
            #print(f'sum1 = {i}')
            for j in range_sum2:
                sum1 = i
                yb_1_num = int(sum1/conc_interval + 1)
                sum2 = j
                yb_2_num = int(sum2/conc_interval + 1)
                range_yb1 = np.linspace(0,sum1, yb_1_num)
                range_yb2 = np.linspace(0,sum2, yb_2_num)
                for yb1 in range_yb1:
                    for yb2 in range_yb2:
                        #df.loc[len(df.index)] = [yb1, sum1-yb1, yb2, sum2-yb2, r]
                        if sum1 == 0:
                            ratio1 = 0.5
                        else:
                            ratio1 = yb1/sum1
                        if sum2 == 0:
                            ratio2 = 0.5
                        else:
                            ratio2 = yb2/sum2
                        pool.append([sum1, ratio1, sum2, ratio2, r/34])
    return pool

def generate_grid_encoded_radius(radius,range_sum1, range_sum2):
    pool = []
    
    for i in range_sum1:
        #print(f'sum1 = {i}')
        for j in range_sum2:
            sum1 = i
            yb_1_num = int(sum1/conc_interval + 1)
            sum2 = j
            yb_2_num = int(sum2/conc_interval + 1)
            range_yb1 = np.linspace(0,sum1, yb_1_num)
            range_yb2 = np.linspace(0,sum2, yb_2_num)
            for yb1 in range_yb1:
                for yb2 in range_yb2:
                    #df.loc[len(df.index)] = [yb1, sum1-yb1, yb2, sum2-yb2, r]
                    if sum1 == 0:
                        ratio1 = 0.5
                    else:
                        ratio1 = yb1/sum1
                    if sum2 == 0:
                        ratio2 = 0.5
                    else:
                        ratio2 = yb2/sum2
                    pool.append([sum1, ratio1, sum2, ratio2, (radius/34)])
    return pool       

In [22]:
pool_encoded = generate_grid_encoded(range_radius,range_sum1, range_sum2)

radius = 5
radius = 7
radius = 9
radius = 11
radius = 13
radius = 15
radius = 17
radius = 19
radius = 21
radius = 23
radius = 25
radius = 27
radius = 29
radius = 31
radius = 33


In [ ]:
pool_encoded = torch.FloatTensor(pool_encoded)
pickle.dump(pool_encoded, open( "NP_pool_medium_conc1_radi2_397528560NP_encoded.pkl", "wb" ) )

In [20]:
import pickle
filename='NP_pool_medium_conc1_radi2_397528560NP_encoded.pkl'
for r in range_radius[:1]:
    t0=time.time()
    print(f'Generating radius ={r}')
    pool_r = generate_grid_encoded_radius(r,range_sum1, range_sum2)
    t1 = time.time()
    print(f'Took {t1-t0} s')
    print(f'Tensorizing')
    pool_rr = torch.FloatTensor(pool_r)
    t2 = time.time()
    print(f'Took {t2-t1} s')
    print(f'Dumping to pickle')
    with open(filename, 'wb') as fp:
        pickle.dump(pool_rr,fp)
    t3 = time.time()
    print(f'Took {t3-t2} s')
        

Generating radius =5
Took 607.4734437465668 s
Tensorizing
Took 7.964658260345459 s
Dumping to pickle
Took 1.0307667255401611 s


In [2]:
import argparse
from datetime import datetime
import math
import numpy as np
import pandas as pd
import time
import torch
import csv


from fireworks import LaunchPad
from jobflow import JobStore
from jobflow.managers.fireworks import flow_to_workflow
from maggma.stores.mongolike import MongoStore
from NanoParticleTools.flows.flows import get_npmc_flow
from NanoParticleTools.inputs.nanoparticle import SphericalConstraint
import uuid

from botorch.models import SingleTaskGP, ModelListGP
from botorch import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qUpperConfidenceBound
from botorch.sampling.samplers import SobolQMCNormalSampler

from botorch.optim import optimize_acqf

from common import seed_generator, configs

from common.utils import get_int, get_qe
from common import utils

def get_data_botorch(data_file, from_cloud = True):
    if from_cloud:
        df = gcloud_utils.get_df_gspread(GSPREAD_CRED, GSPREAD_NAME)
        #df = df.drop(labels=range(1, 600), axis=0)
        my_data = df.to_numpy()
        print(f"reading data log from google sheet: {GSPREAD_NAME}!")
    else:
        my_data = np.loadtxt(data_file, delimiter=',', skiprows=1)
        print(f"reading data log from local: {data_file}!")

    # features
    train_x = torch.from_numpy(my_data[:, :5])
    # labels
    train_y = torch.from_numpy(my_data[:, 5]).unsqueeze(-1)
    # best observation
    best_y = train_y.max().item()
    
    return train_x, train_y, best_y


def encode_inputs(x_arr, x_max = 34):
    '''encode simulation input to botorch'''
    for i, arr in enumerate(x_arr):
        x_arr[i, 0] = arr[0] + arr[1]
        if arr[0] + arr[1] == 0:
            x_arr[i, 1] = 0.5
        else:
            x_arr[i, 1] = arr[0] / (arr[0] + arr[1])
        x_arr[i, 2] = arr[2] + arr[3]
        if arr[2] + arr[3] == 0:
            x_arr[i, 3] = 0.5
        else:
            x_arr[i, 3] = arr[2] / (arr[2] + arr[3])
        x_arr[i, 4] = arr[4] / x_max


def decode_candidates(x_arr, x_max = 34):
    '''decode botorch recommendation candidates for simulation'''
    for i, arr in enumerate(x_arr):
        x_arr[i, 0], x_arr[i, 1] = arr[0] * arr[1], arr[0] * (1 - arr[1])
        x_arr[i, 2], x_arr[i, 3] = arr[2] * arr[3], arr[2] * (1 - arr[3])
        x_arr[i, 4] = arr[4] * x_max

def recommend(train_x, train_y, best_y, bounds, n_trails = 5):
    if isinstance(bounds, list):
        bounds = torch.tensor(bounds)
    elif torch.is_tensor(bounds):
        pass
    else:
        raise TypeError(f"expect bounds in a list or tensor. was given {type(bounds)}")
    
    single_model = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    
    # Expected Improvement acquisition function
    EI = qExpectedImprovement(model = single_model, best_f = best_y)
    # Upper Confidence Bound acquisition function
    UCB = UpperConfidenceBound(single_model, beta=100)
    
    # hyperparameters are super sensitive here
    candidates, _ = optimize_acqf(acq_function = UCB,
                                 bounds = bounds, 
                                 q = n_trails, 
                                 num_restarts = 20, 
                                 raw_samples = 512, 
                                # options = {'batch_limit': 5, "maxiter": 200}
                                 )
    
    return candidates

# New functions for pool-based AL. Different from real-test AL.

def recommend_next(train_X,train_Y,test_X, beta):
    single_model = SingleTaskGP(train_X,train_Y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    UCB = UpperConfidenceBound(single_model, beta)

    ucb = UCB(torch.unsqueeze(test_X,1))
    max_idx = np.argmax(ucb.detach().numpy())
    max_ucb = max(ucb.detach().numpy())
    if max_ucb != ucb.detach().numpy()[max_idx]:
        print('wrong index')
    return max_idx, test_X[max_idx], test_y[max_idx]

def recommend_grid(train_X,train_Y,test_X, beta):
    single_model = SingleTaskGP(train_X,train_Y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    UCB = UpperConfidenceBound(single_model, beta)
    ucb = UCB(torch.unsqueeze(test_X,1))
    max_idx = np.argmax(ucb.detach().numpy())
    max_ucb = max(ucb.detach().numpy())
    if max_ucb != ucb.detach().numpy()[max_idx]:
        print('wrong index')
    return max_idx, test_X[max_idx]

def acq_section(model,test_X, beta):
    UCB = UpperConfidenceBound(model, beta)
    ucb = UCB(torch.unsqueeze(test_X,1))
    max_idx = np.argmax(ucb.detach().numpy())
    max_ucb = max(ucb.detach().numpy())
    if max_ucb != ucb.detach().numpy()[max_idx]:
        print('wrong index')
    return max_idx, test_X[max_idx]
    
def recommend_Qgrid(train_X,train_Y,test_X, beta):
    single_model = SingleTaskGP(train_X,train_Y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    sampler = SobolQMCNormalSampler(1024)
    qUCB = qUpperConfidenceBound(single_model, beta, sampler)
    qucb = qUCB(torch.unsqueeze(test_X,1))
    max_idx = np.argmax(qucb.detach().numpy())
    max_ucb = max(qucb.detach().numpy())
    if max_ucb != qucb.detach().numpy()[max_idx]:
        print('wrong index')
    return max_idx, test_X[max_idx]

# log results
def log_result(test_X,test_Y,max_idx,data_dest):
    decode_candidates(test_X)
    features = test_X[max_idx].numpy()
    obs = test_Y[max_idx].numpy()
    df_features = pd.DataFrame(features.reshape(-1, len(features)))
    df_obs = pd.DataFrame(obs)
    df_log = pd.concat([df_features,df_obs],axis=1)
    df_log.columns =['yb_1', 'er_1', 'yb_2', 'er_2','radius','UV']
    log = pd.read_csv(data_dest)
    log = log.append(df_log, ignore_index=True)
    log.to_csv(data_dest, index=False)

def append_result(test_X,test_Y,max_idx,log):
    decode_candidates(test_X)
    features = test_X[max_idx].numpy()
    obs = test_Y[max_idx].numpy()
    result = np.concatenate((features, obs), axis=None)
    log.append(result)
    
# update pool
def update_pool(train_X,train_Y,test_X,test_Y,pool_X, pool_Y, max_idx):
    train_X, train_Y = torch.cat([train_X,torch.unsqueeze(pool_X[max_idx],0)]),torch.cat([train_Y,torch.unsqueeze(pool_Y[max_idx],0)])
    #print(pool_x[max_idx])
    if max_idx ==0:
        pool_X, pool_Y = pool_X[1:], pool_y[1:]
    elif max_idx ==(len(pool_X)-1):
        pool_X, pool_Y = pool_X[:-1], pool_y[:-1]
    else:
        pool_X, pool_Y = torch.cat([pool_X[:max_idx], pool_X[max_idx+1:]]), torch.cat([pool_Y[0:max_idx], pool_Y[max_idx+1:]]),
    test_X, test_Y = torch.clone(pool_X), torch.clone(pool_Y)
    return train_X,train_Y,test_X,test_Y,pool_X, pool_Y

def add_one_ensemble(log_seed,i):
    df = pd.DataFrame((log_seed[i]))
    df.columns =['yb_1', 'er_1', 'yb_2', 'er_2','radius','UV']

    evalued = len(df)
    max_uv = []
    y = df.UV
    for i in range(evalued):
        max_uv.append(df.iloc[0:i+1].UV.max())
    x = range(len(max_uv))
    max_x = len(max_uv)
    max_y = max(max_uv)    
    ax1.scatter(x, y, c='black', alpha = 0.3)
    ax2.plot(x, max_uv, c='r', alpha = 1)
    return max_x, max_y


[KeOps] Warning : omp.h header is not in the path, disabling OpenMP.
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


<stdin>:1:10: fatal error: 'omp.h' file not found
#include <omp.h>
         ^~~~~~~
1 error generated.


In [3]:
# read from cPickle
import pickle
pool_x = pickle.load( open( "../saved_data/NP_pool_small_conc2_radi2_26334375NP_encoded.pkl", "rb" ) )
train_x, train_y, _ = get_data_botorch("../saved_data/UV_log_shuffled_10initial_test1_grid_beta=10000.csv", from_cloud=False)

reading data log from local: ../saved_data/UV_log_shuffled_10initial_test1_grid_beta=10000.csv!


In [112]:
train_x, train_y, _ = get_data_botorch("../saved_data/UV_log_shuffled_10initial_test1_grid_beta=10000.csv", from_cloud=False)
encode_inputs(train_x)
def recommend_grid_2steps(train_X, train_Y, test_X, pie_size=100000, beta=10000):
    # split the original pool to sectors that contains 10,000 grids in each sector
    pies = torch.split(test_X, pie_size)
    idx_list = []
    single_model = SingleTaskGP(train_X,train_Y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)

    # the candidates from each sector will be saved in 'seleced'
    selected = torch.empty((0,5))
    for i,pool_i in enumerate(pies):
        #print(f'recommending section {i}/{len(pie)} ......')
        max_idx, max_feature = acq_section(single_model, pool_i, beta)
        selected = torch.cat((selected,torch.unsqueeze(max_feature,0)),0)
        idx_list.append(i*pie_size+max_idx)

    # the final round of recommedation from the candidataes in the selected pool
    max_idx_final, max_feature_final = acq_section(single_model, selected, beta)
    return torch.unsqueeze(max_feature_final,0), idx_list[max_idx_final]

reading data log from local: ../saved_data/UV_log_shuffled_10initial_test1_grid_beta=10000.csv!


In [113]:
candidate, idx = recommend_grid_2steps(train_x, train_y, pool_x, pie_size=100000, beta=10000)

In [52]:
decode_candidates(candidate)

In [114]:
candidate

tensor([[0.9200, 0.9787, 0.0000, 0.5000, 0.9706]])

In [115]:
idx

26009795

In [116]:
pool_x[idx]

tensor([0.9200, 0.9787, 0.0000, 0.5000, 0.9706])

In [117]:
def update_grid(pool_X, max_idx):
    if max_idx ==0:
        pool_X = pool_X[1:]
    elif max_idx ==(len(pool_X)-1):
        pool_X = pool_X[:-1]
    else:
        pool_X = torch.cat([pool_X[:max_idx], pool_X[max_idx+1:]])    
    return pool_X

In [118]:
pool_x = update_grid(pool_x, idx)

In [119]:
pool_x[idx]

tensor([0.9200, 1.0000, 0.0000, 0.5000, 0.9706])

In [ ]:
pickle.dump(pool_x, open( "../saved_data/NP_pool_small_conc2_radi2_26334375NP_encoded.pkl", "wb" ) )